In [1]:
import pandas as pd 
import numpy as np
import xgboost
from sklearn.preprocessing import OneHotEncoder
from pandas_profiling import ProfileReport
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")
alldata = train.append(test)

In [3]:
alldata

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [4]:
print(train.shape,test.shape,alldata.shape)

(891, 12) (418, 11) (1309, 12)


In [5]:
print(alldata.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [6]:
alldata.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [7]:
alldata.dtypes

PassengerId      int64
Survived       float64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [8]:
profile = ProfileReport(alldata, title="Pandas Profiling Report")
profile.to_widgets()


In [9]:
alldataX = alldata[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
alldatay = train[["Survived"]]

In [10]:
PclassDumm = pd.get_dummies(alldata.Pclass, prefix='Pclass')
PclassDumm.head(10)

,Pclass_1,Pclass_2,Pclass_3
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1
5,0,0,1
6,1,0,0
7,0,0,1
8,0,0,1
9,0,1,0


In [11]:
SexDumm = pd.get_dummies(alldata.Sex, prefix='Sex')
SexDumm.head(10)

,Sex_female,Sex_male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
5,0,1
6,0,1
7,0,1
8,1,0
9,1,0


In [12]:
SibSpDumm = pd.get_dummies(alldata.SibSp, prefix='SibSp')
SibSpDumm.head(10)

,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0
5,1,0,0,0,0,0,0
6,1,0,0,0,0,0,0
7,0,0,0,1,0,0,0
8,1,0,0,0,0,0,0
9,0,1,0,0,0,0,0


In [13]:
ParchDumm = pd.get_dummies(alldata.Parch, prefix='Parch')
ParchDumm.head(10)

,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9
0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0
9,1,0,0,0,0,0,0,0


In [14]:
EmbarkedDumm = pd.get_dummies(alldata.Embarked, prefix='Embarked')
EmbarkedDumm.head(10)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0
6,0,0,1
7,0,0,1
8,0,0,1
9,1,0,0


In [15]:
alldata = pd.concat([alldata, PclassDumm, SexDumm, SibSpDumm, ParchDumm, EmbarkedDumm], axis=1, ignore_index=False)
alldata

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,0,0,0,0,0,0,0,0,0,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,0,0,0,0,0,0,1,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,0,0,0,0,0,0,0,0,0,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,0,0,0,0,0,0,0,0,0,1
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,...,0,0,0,0,0,0,0,0,0,1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,...,0,0,0,0,0,0,0,1,0,0
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,...,0,0,0,0,0,0,0,0,0,1
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,...,0,0,0,0,0,0,0,0,0,1


In [16]:
alldata = alldata.drop(labels=["Name","Ticket","PassengerId","Cabin","Pclass","Sex","SibSp","Parch","Embarked"], axis = 1)

In [17]:
alldata

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,...,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9,Embarked_C,Embarked_Q,Embarked_S
0,0.0,22.0,7.2500,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1.0,38.0,71.2833,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,1.0,26.0,7.9250,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,1.0,35.0,53.1000,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0.0,35.0,8.0500,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,NaN,8.0500,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
414,NaN,39.0,108.9000,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
415,NaN,38.5,7.2500,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
416,NaN,NaN,8.0500,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
trainX = alldata[alldata["Survived"].notnull()].drop(["Survived"],axis = 1)

In [19]:
trainX

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,...,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,38.0,71.2833,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,26.0,7.9250,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,35.0,53.1000,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,35.0,8.0500,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.0,13.0000,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
887,19.0,30.0000,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
888,NaN,23.4500,0,0,1,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
889,26.0,30.0000,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
trainy = alldata.Survived[alldata.Survived.notnull()]

In [21]:
trainy

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
886    0.0
887    1.0
888    0.0
889    1.0
890    0.0
Name: Survived, Length: 891, dtype: float64

In [22]:
testX = alldata[alldata["Survived"].isnull()].drop(["Survived"],axis = 1)

In [23]:
testX

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,...,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9,Embarked_C,Embarked_Q,Embarked_S
0,34.5,7.8292,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,47.0,7.0000,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,62.0,9.6875,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,27.0,8.6625,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,22.0,12.2875,0,0,1,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,8.0500,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
414,39.0,108.9000,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
415,38.5,7.2500,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
416,NaN,8.0500,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
null_columns=trainX.columns[trainX.isnull().any()]
trainX[null_columns].isnull().sum()

Age    177
dtype: int64

In [25]:
null_columns=testX.columns[testX.isnull().any()]
testX[null_columns].isnull().sum()

Age     86
Fare     1
dtype: int64

In [26]:
trainX["Age"] = trainX["Age"].fillna(trainX["Age"].mean())

In [27]:
null_columns=trainX.columns[trainX.isnull().any()]
trainX[null_columns].isnull().sum()

Series([], dtype: float64)

In [28]:
trainX.Age

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

In [29]:
testX["Age"] = testX["Age"].fillna(testX["Age"].mean())
testX["Fare"] = testX["Fare"].fillna(testX["Fare"].mean())

In [30]:
null_columns=testX.columns[testX.isnull().any()]
testX[null_columns].isnull().sum()

Series([], dtype: float64)

In [31]:
model = XGBClassifier()
model.fit(trainX, trainy)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
testpred = model.predict(testX)

In [33]:
testpred

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0.

In [34]:
testpred=pd.DataFrame(testpred)

In [35]:
testpred

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
413,0.0
414,1.0
415,0.0
416,0.0


In [36]:
testpred.to_csv("Data/Submission.csv")

In [37]:
from tpot import TPOTClassifier
pipeline_optimizer = TPOTClassifier(generations=100, population_size=20, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(trainX, trainy)

C:\ProgramData\Anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")



Generation 1 - Current best internal CV score: 0.8204820789655388
Generation 2 - Current best internal CV score: 0.8204820789655388
Generation 3 - Current best internal CV score: 0.8204820789655388
Generation 4 - Current best internal CV score: 0.8204820789655388
Generation 5 - Current best internal CV score: 0.8272048207896553
Generation 6 - Current best internal CV score: 0.8272048207896553
Generation 7 - Current best internal CV score: 0.8272048207896553
Generation 8 - Current best internal CV score: 0.8305630531667816
Generation 9 - Current best internal CV score: 0.8305630531667816
Generation 10 - Current best internal CV score: 0.8305756073065093
Generation 11 - Current best internal CV score: 0.8316740945326722
Generation 12 - Current best internal CV score: 0.8339463938233631
Generation 13 - Current best internal CV score: 0.8339463938233631
Generation 14 - Current best internal CV score: 0.8339463938233631
Generation 15 - Current best internal CV score: 0.8339463938233631
Gen

TPOTClassifier(log_file=<ipykernel.iostream.OutStream object at 0x00000200A28257C8>,
               population_size=20, random_state=42, verbosity=2)

In [38]:
testpred = pipeline_optimizer.predict(testX)

In [39]:
testpred = pd.DataFrame(testpred)
testpred.to_csv("Data/Submission3.csv")